# Blank Grids

In [ ]:
import json
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os

# Ensure the output directory exists
output_directory = 'blank_grids'
os.makedirs(output_directory, exist_ok=True)

def create_grid_image(rows, cols, image_size=1000, metadata_file='metadata.json'):
    # Create a new white image
    for line_width in [10, 20]:
        img = Image.new('RGB', (image_size, image_size), 'white')
        draw = ImageDraw.Draw(img)

        # Calculate the width of each cell
        cell_width = (image_size - line_width) / cols
        cell_height = (image_size - line_width) / rows

        # Draw vertical lines
        for i in range(cols + 1):
            x = i * cell_width + line_width / 2
            draw.line([(x, 0), (x, image_size)], fill='black', width=line_width)

        # Draw horizontal lines
        for i in range(rows + 1):
            y = i * cell_height + line_width / 2
            draw.line([(0, y), (image_size, y)], fill='black', width=line_width)

        # Draw the border lines
        draw.line([(0, 0), (image_size, 0)], fill='black', width=line_width)  # Top border
        draw.line([(0, 0), (0, image_size)], fill='black', width=line_width)  # Left border
        draw.line([(0, image_size - line_width / 2), (image_size, image_size - line_width / 2)], fill='black', width=line_width)  # Bottom border
        draw.line([(image_size - line_width / 2, 0), (image_size - line_width / 2, image_size)], fill='black', width=line_width)  # Right border

        im_size = [2000, 1250, 500]

        for s in im_size:
            image_name = f'grid_{rows}x{cols}_{s}_{line_width}.png'
            image_path = os.path.join(output_directory, image_name)
            img.resize((s, s)).save(image_path)

            # Create metadata
            metadata = {
                image_name: {
                    'rows': rows,
                    'columns': cols,
                    'cells': rows * cols,
                    'image_size': f'{s}x{s}',
                    'linewidth': line_width
                }
            }

            # Load existing metadata if file exists
            metadata_path = os.path.join(output_directory, metadata_file)
            try:
                if os.path.exists(metadata_path):
                    with open(metadata_path, 'r') as f:
                        all_metadata = json.load(f)
                else:
                    all_metadata = {}
            except json.JSONDecodeError:
                all_metadata = {}

            # Append new metadata
            all_metadata.update(metadata)

            # Save updated metadata to JSON file
            with open(metadata_path, 'w') as f:
                json.dump(all_metadata, f, indent=4)

# Generate all combinations of images from 3x3 to 15x15 with different DPIs
resolutions = [2000]
for size in range(3, 16):
    for rows in range(size, size + 2):
        for resolution in resolutions:
            if rows <= 15:
                cols = size
                create_grid_image(rows, cols, image_size=resolution)
                if rows != cols:
                    create_grid_image(cols, rows, image_size=resolution)


# Word grids

In [ ]:
import matplotlib.pyplot as plt
import os

In [ ]:
import json
import os
from PIL import Image, ImageDraw, ImageFont

# Ensure the output directory exists
output_directory = 'word_grids'
os.makedirs(output_directory, exist_ok=True)

# Provided list of words
words = [
    "apple", "book", "car", "door", "earth", "fruit", "game", "house", "ice", "joke",
    "kite", "lamp", "moon", "night", "orange", "pen", "queen", "rain", "ship", "tree",
    "uncle", "voice", "water", "x-ray", "yarn", "zebra", "army", "baby", "cake", "dance",
    "echo", "flag", "green", "happy", "idea", "jelly", "king", "lion", "mouse", "north",
    "opera", "plant", "quiet", "robot", "stone", "tiger", "union", "vivid", "world", "youth",
    "zoo", "brave", "climb", "dream", "alert", "bread", "clean", "drama", "elbow", "flame",
    "grape", "heart", "index", "juice", "knees", "lemon", "march", "noble", "ocean", "peace",
    "quake", "reign", "shine", "track", "unity", "vapor", "weave", "young", "zone", "angle",
    "bloom", "crisp", "diver", "equip", "fable", "giant", "hover", "ideal", "jolly", "knock",
    "lunar", "march", "neigh", "orbit", "plush", "quote", "ridge", "skate", "truth", "urban",
    "valve", "whale", "hedge", "yield", "azure", "blaze", "charm", "donor", "eagle", "flock",
    "grain", "hinge", "inbox", "jewel", "kneel", "layer", "metal", "nurse", "octet", "piano",
    "quilt", "reach", "slope", "trace", "usher", "villa", "whisk", "xerox", "yacht", "bingo",
    "craft", "drill", "event", "flora", "globe", "haven", "image", "jumpy", "koala", "lodge",
    "mango", "nerve", "offer", "pouch", "realm", "salad", "tease", "ultra", "vigor", "wrist",
    "xylol", "yogic", "zonal", "blend", "crane", "diner", "elude", "forge", "glint", "hover",
    "icily", "joker", "knead", "laser", "medal", "nasal", "optic", "pluck", "royal", "steep",
    "tulip", "udder", "viper", "wafer", "xerox", "yield", "abode", "brisk", "cider", "delta",
    "frown", "gripe", "haste", "inlet", "jolly", "knock", "leaky", "manor", "naval", "oxide",
    "prone", "roast", "scout", "trunk", "unzip", "vogue", "weary", "xerox", "yodel", "amber",
    "brace", "creek", "drift", "enter", "facet", "grape", "hoist", "inbox", "joint", "karma",
    "latch", "modem", "nerve", "orbit", "pivot", "query", "rough", "stark", "tread", "urban",
    "value", "whack", "xenon", "youth", "azure", "blurt", "chess", "drape", "ebony", "flair",
    "grill", "haste", "inter", "jiffy", "khaki", "ledge", "meaty", "nudge", "octal", "polka",
    "quack", "relax", "slant", "tenth", "undue", "verge", "waver", "xenon", "yacht", "adobe",
    "bongo", "crock", "diary", "edict", "flute", "graft", "hippo", "imply", "joker", "knave"
]

# Path to the downloaded font
font_path = "LiberationSans-Regular.ttf"

def get_word_for_cell(row, col, cols):
    index = row * cols + col
    return words[index % len(words)]

def create_grid_image(rows, cols, image_size=2000, metadata_file='word_grids_metadata.json'):
    # Create a new white image
    for line_width in [10, 20]:
        img = Image.new('RGB', (image_size, image_size), 'white')
        draw = ImageDraw.Draw(img)

        # Calculate the width of each cell
        cell_width = (image_size - line_width * (cols + 1)) / cols
        cell_height = (image_size - line_width * (rows + 1)) / rows

        # Base font size calculation
        base_font_size = int(min(cell_width, cell_height) * 0.2)

        # Draw vertical lines
        for i in range(cols + 1):
            x = i * cell_width + i * line_width + line_width / 2
            draw.line([(x, 0), (x, image_size)], fill='black', width=line_width)

        # Draw horizontal lines
        for i in range(rows + 1):
            y = i * cell_height + i * line_width + line_width / 2
            draw.line([(0, y), (image_size, y)], fill='black', width=line_width)

        # Draw the border lines
        draw.line([(0, 0), (image_size, 0)], fill='black', width=line_width)  # Top border
        draw.line([(0, 0), (0, image_size)], fill='black', width=line_width)  # Left border
        draw.line([(0, image_size - line_width / 2), (image_size, image_size - line_width / 2)], fill='black', width=line_width)  # Bottom border
        draw.line([(image_size - line_width / 2, 0), (image_size - line_width / 2, image_size)], fill='black', width=line_width)  # Right border

        # Initialize metadata for cell words
        cell_words = {}

        # Draw a unique word in each cell, adjusting font size based on cell size
        for row in range(rows):
            for col in range(cols):
                word = get_word_for_cell(row, col, cols)
                cell_words[f'{row},{col}'] = word
                font_size = int(min(cell_width, cell_height) * 0.2)

                try:
                    font = ImageFont.truetype(font_path, font_size)
                except IOError:
                    font = ImageFont.load_default()

                text_bbox = draw.textbbox((0, 0), word, font=font)
                text_width = text_bbox[2] - text_bbox[0]
                text_height = text_bbox[3] - text_bbox[1]

                if (image_size < 1000):
                    x_multiplier = ((((rows+cols)//2)**1.71)*0.2)/(cell_width*0.13)
                    text_x = x_multiplier + col * cell_width + col * line_width + (cell_width - text_width) / 2 + line_width / 2

                    y_multiplier = ((((rows+cols)//2)**1.71)*0.2)/(cell_width*0.15)
                    text_y = y_multiplier + row * cell_height + row * line_width + (cell_height - text_height) / 2 + line_width / 2
                else:
                    text_x = cell_width*0.04 + col * cell_width + col * line_width + (cell_width - text_width) / 2 + line_width / 2
                    text_y = row * cell_height + row * line_width + (cell_height - text_height) / 2 + line_width / 2

                draw.text((text_x, text_y), word, fill='black', font=font)

        im_size = [2000, 1250, 500]

        for s in im_size:
            image_name = f'grid_{rows}x{cols}_{s}_{line_width}.png'
            image_path = os.path.join(output_directory, image_name)
            img.resize((s, s)).save(image_path)

            # Create metadata
            metadata = {
                image_name: {
                    'rows': rows,
                    'columns': cols,
                    'cells': rows * cols,
                    'image_size': f'{s}x{s}',
                    'cell_words': cell_words,
                    'linewidth': line_width
                }
            }

            # Load existing metadata if file exists
            metadata_path = os.path.join(output_directory, metadata_file)
            try:
                if os.path.exists(metadata_path):
                    with open(metadata_path, 'r') as f:
                        all_metadata = json.load(f)
                else:
                    all_metadata = {}
            except json.JSONDecodeError:
                all_metadata = {}

            # Append new metadata
            all_metadata.update(metadata)

            # Save updated metadata to JSON file
            with open(metadata_path, 'w') as f:
                json.dump(all_metadata, f, indent=4)

# Generate the specified combinations of images with words in each cell
resolutions = [500, 1250, 2000]

for size in range(3, 16):
    for rows in range(size, size + 2):
        if rows <= 15:
            cols = size
            for resolution in resolutions:
                create_grid_image(rows, cols, image_size=resolution)
                if rows != cols:
                    create_grid_image(cols, rows, image_size=resolution)